In [ ]:
import nltk
import openpyxl
import pymorphy3
import json
import numpy as np
import pandas as pd
import datetime
import sklearn

In [ ]:
import datetime

def string_date_parse(data):
  return datetime.datetime.strptime(str(data), '%Y%m%d').date()

In [ ]:
market_directory = 'market_data'
market_files = {
    'sber':'SBER.csv',
    'lukoil':'LKOH.csv',
    'gazprom':'GAZP.csv',
    'moex_i':'MOEX.csv'
}

In [ ]:
market_data = {}

In [ ]:
for key in market_files.keys():
  temp = pd.read_csv(f"{market_directory}/{market_files[key]}")
  # удаляем ненужные символы из названия колонок
  temp.columns = [elem.replace('<','').replace('>','') for elem in temp.columns]
  # удаляем ненужные столбцы
  temp = temp.drop(['PER', 'TIME'], axis=1)

  # среднее значение между максимумом и минимумом
  mid = temp.apply(lambda data: (data['HIGH'] + data['LOW'])/2, axis=1)
  # вставляем столбец со средним значением
  temp.insert(list(temp.columns).index('HIGH')+1, 'MID', mid)

  # преобразуем дату к datetime
  temp['DATE'] = temp['DATE'].apply(lambda data: string_date_parse(data))

  market_data[key] = temp

In [ ]:
market_data['moex_i']

,TICKER,DATE,OPEN,HIGH,MID,LOW,CLOSE,VOL
0,IMOEX,2000-04-03,257.30,257.30,257.300,257.30,257.30,0
1,IMOEX,2000-04-04,258.02,258.02,258.020,258.02,258.02,0
2,IMOEX,2000-04-05,245.70,245.70,245.700,245.70,245.70,0
3,IMOEX,2000-04-06,249.43,249.43,249.430,249.43,249.43,0
4,IMOEX,2000-04-07,247.96,247.96,247.960,247.96,247.96,0
...,...,...,...,...,...,...,...,...
5741,IMOEX,2023-04-05,2489.18,2500.80,2483.370,2465.94,2500.80,46244692124
5742,IMOEX,2023-04-06,2505.18,2521.65,2509.975,2498.30,2498.30,43297539433
5743,IMOEX,2023-04-07,2505.22,2511.84,2501.605,2491.37,2508.39,28843962457
5744,IMOEX,2023-04-10,2524.67,2548.29,2536.020,2523.75,2547.35,50841711389


In [ ]:
def getDateOnly(date):
  return datetime.date(date.year, date.month, date.day)

In [ ]:
import math
def interpolate_data(data_start, data_end):
  ticker = data_start['TICKER']
  current_date = data_start['DATE']
  days_number = (getDateOnly(data_end['DATE'])-getDateOnly(data_start['DATE'])).days
  data_delta = data_start.copy()

  del data_delta['DATE']
  del data_delta['TICKER']

  for key in data_delta.keys():
    data_delta[key] = (data_end[key] - data_start[key])/days_number

  df = []
  for i in range(1, days_number):
    temp = data_delta.copy()

    for key in temp.keys():
      temp[key] = data_start[key] + temp[key]*i

    temp['VOL'] = int(math.floor(temp['VOL']))
    temp['DATE'] = (getDateOnly(data_start['DATE']) + datetime.timedelta(days=i))
    temp['TICKER'] = ticker

    df.append(temp)

  return df

Интерполируем недостающие даты: значения будут равны средним значениям

In [ ]:
def market_interpolate(df):
  n = len(df)
  i = 1
  while i < n:
    if (getDateOnly(df.iloc[i]['DATE']) - getDateOnly(df.iloc[i - 1]['DATE'])).days > 1:
      new_data = interpolate_data(df.iloc[i - 1], df.iloc[i])
      interpolated = pd.DataFrame(new_data)
      df = pd.concat([df.iloc[:i], interpolated, df.iloc[i:]]).reset_index(drop=True)
      n = len(df)
    i += 1

  return df

In [ ]:
date_default = market_data['sber']['DATE']
date_default_set = {x for x in market_data['sber']['DATE']}

In [ ]:
for key in market_data.keys():
  market_data[key] = market_interpolate(market_data[key])

In [ ]:
date_converted = market_data['sber']['DATE']
date_converted_set = {x for x in market_data['sber']['DATE']}
len(date_converted)

8409

In [ ]:
date_converted_set.difference(date_default_set)

## Финальные предсказания 

In [ ]:
def getYWithDate(stocks):
  y = {'DATE':[], 'Y':[]}
  n = len(stocks)
  for i in range(1, n):
    data = stocks.loc[i]
    prev_data = stocks.loc[i-1]
    percent_delta = (data['MID'] - prev_data['MID'])/prev_data['MID']
    
    y['DATE'].append(data['DATE'])
    y['Y'].append(percent_delta)
  return y  

In [ ]:
market_data['moex_i']

,TICKER,DATE,OPEN,HIGH,MID,LOW,CLOSE,VOL
0,IMOEX,2000-04-03,257.300000,257.30,257.300000,257.300000,257.300000,0
1,IMOEX,2000-04-04,258.020000,258.02,258.020000,258.020000,258.020000,0
2,IMOEX,2000-04-05,245.700000,245.70,245.700000,245.700000,245.700000,0
3,IMOEX,2000-04-06,249.430000,249.43,249.430000,249.430000,249.430000,0
4,IMOEX,2000-04-07,247.960000,247.96,247.960000,247.960000,247.960000,0
...,...,...,...,...,...,...,...,...
8404,IMOEX,2023-04-07,2505.220000,2511.84,2501.605000,2491.370000,2508.390000,28843962457
8405,IMOEX,2023-04-08,2511.703333,2523.99,2513.076667,2502.163333,2521.376667,36176545434
8406,IMOEX,2023-04-09,2518.186667,2536.14,2524.548333,2512.956667,2534.363333,43509128411
8407,IMOEX,2023-04-10,2524.670000,2548.29,2536.020000,2523.750000,2547.350000,50841711389


In [ ]:
sberY = pd.DataFrame(getYWithDate(market_data['moex_i']),columns=['DATE', 'Y'])
sberY

,DATE,Y
0,2000-04-04,0.002798
1,2000-04-05,-0.047748
2,2000-04-06,0.015181
3,2000-04-07,-0.005893
4,2000-04-08,0.007152
...,...,...
8403,2023-04-07,-0.003335
8404,2023-04-08,0.004586
8405,2023-04-09,0.004565
8406,2023-04-10,0.004544


In [ ]:
def getDateDiffs(dates):
  dates.values.tolist()
  result = []
  for i in range(1, len(dates)):
    result.append((getDateOnly(dates[i]) - getDateOnly(dates[i-1])).days)
  df = pd.DataFrame(result, columns=['DATE_DIFF'])
  return df.groupby(['DATE_DIFF'])['DATE_DIFF'].count()

смотрим пропущенные дни. Число больше единицы означает пропуск в n-1 день

In [ ]:
getDateDiffs(date_default)

DATE_DIFF
1     4497
2       57
3     1045
4       97
5       22
6        4
7        1
11       5
12       3
15       1
27       1
Name: DATE_DIFF, dtype: int64

In [ ]:
getDateDiffs(date_converted)

DATE_DIFF
1    8408
Name: DATE_DIFF, dtype: int64

# Сохраняем предсказания для всех существующих ценных бумаг

In [ ]:
pd.DataFrame(getYWithDate(market_data['sber']),columns=['DATE', 'Y'])

,DATE,Y
0,2000-04-04,0.046203
1,2000-04-05,-0.008045
2,2000-04-06,-0.008459
3,2000-04-07,-0.020185
4,2000-04-08,0.015986
...,...,...
8403,2023-04-07,-0.003268
8404,2023-04-08,0.006844
8405,2023-04-09,0.006798
8406,2023-04-10,0.006752


In [ ]:
market_data.keys()

dict_keys(['sber', 'lukoil', 'gazprom', 'moex_i'])

сохранение предсказаний Y в виде процета изменения стоимости по отношению к предыдущей дате

In [ ]:
for key in market_data.keys():
  pd.DataFrame(getYWithDate(market_data[key]),columns=['DATE', 'Y']).to_csv(f'./market_data/{key}_Y.csv', index=False)

сохранение полного датасета

In [ ]:
for key in market_data.keys():
  pd.DataFrame(market_data[key]).to_csv(f'./market_data/{key}_FULL.csv', index=False)